In [2]:
!pip install python_speech_features

  Using cached python_speech_features-0.6.tar.gz (5.6 kB)
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=1fe6e783d45e574b3cb575f4559d52c16cca518fd8495c254bbb3720037864d2
  Stored in directory: c:\users\91922\appdata\local\pip\cache\wheels\09\a1\04\08e2688d2562d8f9ff89e77c6ddfbf7268e07dae1a6f22455e
Successfully built python-speech-features


In [1]:

from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile

import os
import pickle
import random
import operator

import math


In [4]:
# function to get the distance between feature vecotrs and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [5]:
# identify the class of the instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

In [2]:
# function to evaluate the model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [3]:
# directory that holds the wav files
# directory = r"C://Users//vedan//genres//"
directory = r"C://Users//91922//Documents//19102A0058//BE//ML//ML Project//genres_original//"
# binary file where we will collect all the features extracted using mfcc (Mel Frequency Cepstral Coefficients)
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

In [4]:
# Split the dataset into training and testing sets respectively
dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

In [5]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression()

In [13]:
import pandas as pd
df=pd.DataFrame(trainingSet)
df

,0,1,2
0,"[76.5026110677372, -1.9614173589380144, -15.77...","[[31.364326631614066, -1.3468342058680456, -0....",1
1,"[66.93164964522649, 0.7188534538996809, -3.267...","[[107.79187315415385, -31.41110032880321, -9.2...",1
2,"[78.7741031258866, 3.039700087120104, -19.0244...","[[41.92951882858838, -21.586957755456382, -30....",1
3,"[71.97284566377586, 0.5064100190354653, -23.83...","[[73.97934723712847, -49.0136740941537, -34.35...",1
4,"[70.69884409913618, 1.9345714669240475, -15.94...","[[110.04173712862605, -82.13417756594248, -59....",1
...,...,...,...
649,"[73.39782483055345, -4.459683299978229, -14.65...","[[21.58115076126584, -17.780006213473275, -4.3...",10
650,"[73.96549448871167, -4.293810209685265, -18.14...","[[36.05839587905571, -21.803887976795558, -7.0...",10
651,"[75.55902537392254, -5.658250351745185, -24.03...","[[18.18242292339917, -15.896549001368676, -7.1...",10
652,"[65.61635032690315, 2.2903386914587833, -10.02...","[[122.25111662065376, -35.56627502776603, -71....",10


In [14]:
trainingSet

[(array([ 76.50261107,  -1.96141736, -15.7743469 ,   3.8314189 ,
         -10.47332553,   1.31182833, -19.39373183,   5.28678994,
         -16.63172467,   5.3534445 ,  -9.85657026,   6.25007663,
          -5.58749505]),
  array([[ 3.13643266e+01, -1.34683421e+00, -9.48272203e-01,
          -2.05382281e+01, -1.45847814e+01, -1.72347943e+01,
          -9.35145739e+00,  2.72034545e+00, -3.29035455e+00,
           1.04057336e+00,  2.32513168e+00,  6.09119695e+00,
           8.44732834e+00],
         [-1.34683421e+00,  3.64751586e+01, -2.19100677e+01,
          -1.52506981e+01, -7.71247217e+00, -1.42477860e+01,
          -6.48177523e+00,  3.95463925e+00,  1.14509487e+00,
          -4.30576920e+00,  9.05817993e+00,  8.98145706e+00,
           7.68495192e+00],
         [-9.48272203e-01, -2.19100677e+01,  7.63578328e+01,
           1.69425823e+01, -2.67611115e+01, -2.71149039e+00,
           1.99049253e+01,  1.64116110e+01, -1.31355709e+01,
           1.53609455e+01, -1.31293962e+01, -7.478976

In [10]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [11]:
# making predictions using KNN
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.7125


In [15]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo

# create the root window
root = tk.Tk()
root.title('Tkinter Open File Dialog')
root.resizable(False, False)
root.geometry('300x150')


def select_file():
    filetypes = (
        ('wav files', '*.wav'),
        ('All files', '*.*')
    )
    global filename
    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='/',
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=filename
    )


# open button
open_button = ttk.Button(
    root,
    text='Open a File',
    command=select_file
)

open_button.pack(expand=True)


# run the application
root.mainloop()

In [16]:
# testing the code with external samples
# URL: https://uweb.engr.arizona.edu/~429rns/audiofiles/audiofiles.html

test_dir = filename
# test_file = test_dir + "test.wav"
test_file = test_dir 
# test_file = test_dir + "test4.wav"

In [17]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

In [19]:
from collections import defaultdict
results = defaultdict(int)

directory = r"C://Users//91922//Documents//19102A0058//BE//ML//ML Project//genres_original//"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [20]:
pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])

jazz


In [21]:
import tkinter 
from tkinter import *

def printSomething():
    # if you want the button to disappear:
    # button.destroy() or button.pack_forget()
    label = Label(root, text= results[pred] )
    #this creates a new label to the GUI
    label.pack() 

root = Tk()

button = Button(root, text="output", command=printSomething) 
button.pack()

root.mainloop()